## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-09-02-22-00 +0000,nypost,OpenAI CEO Sam Altman served subpoena onstage ...,https://nypost.com/2025/11/08/us-news/openai-c...
1,2025-11-09-02-18-31 +0000,startribune,Live: Updates from Minnesota girls volleyball ...,https://www.startribune.com/high-school-volley...
2,2025-11-09-02-15-53 +0000,bbc,US authorities ground cargo plane model after ...,https://www.bbc.com/news/articles/cx2pr5y7p90o...
3,2025-11-09-02-13-38 +0000,startribune,Live: Updates from football state quarterfinals,https://www.startribune.com/scores-photos-and-...
4,2025-11-09-02-07-38 +0000,bbc,Honesty boxes should be dying like cash. But m...,https://www.bbc.com/news/articles/cjd05rj5xl9o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,31
158,snap,9
171,shutdown,7
172,flight,7
324,up,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...,71
182,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...,69
184,2025-11-08-02-59-00 +0000,wsj,Supreme Court Pauses Order for Trump Administr...,https://www.wsj.com/us-news/law/supreme-court-...,68
77,2025-11-08-17-30-00 +0000,wsj,"Middle-class voters, many of them frustrated w...",https://www.wsj.com/politics/elections/democra...,64
174,2025-11-08-05-16-33 +0000,bbc,Trump gives Hungary one-year exemption from Ru...,https://www.bbc.com/news/articles/cp85dvnkpv4o...,58


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,71,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...
85,45,2025-11-08-17-17-26 +0000,nypost,New Yorkers flock to food banks as government ...,https://nypost.com/2025/11/08/us-news/trump-ur...
77,33,2025-11-08-17-30-00 +0000,wsj,"Middle-class voters, many of them frustrated w...",https://www.wsj.com/politics/elections/democra...
102,32,2025-11-08-14-29-08 +0000,cbc,U.S. Federal Aviation Administration grounds a...,https://www.cbc.ca/news/world/ups-fedex-ground...
81,31,2025-11-08-17-23-43 +0000,nypost,"Andrew Mountbatten Windsor, Fergie being probe...",https://nypost.com/2025/11/08/world-news/princ...
80,31,2025-11-08-17-25-55 +0000,nypost,Mamdani-backing Hochul insists she doesn’t ‘su...,https://nypost.com/2025/11/08/us-news/mamdani-...
46,28,2025-11-08-21-59-45 +0000,nypost,"At least 5 dead, hundreds injured after tornad...",https://nypost.com/2025/11/08/world-news/at-le...
29,25,2025-11-08-23-42-25 +0000,startribune,Minnesota restores full SNAP benefits to famil...,https://www.startribune.com/minnesota-restores...
24,22,2025-11-09-00-00-18 +0000,nypost,Hamas claims to have found body of IDF soldier...,https://nypost.com/2025/11/08/world-news/hamas...
122,22,2025-11-08-12-00-00 +0000,nypost,Zohran Mamdani’s wife Rama Duwaji uses her art...,https://nypost.com/2025/11/08/us-news/zohran-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
